# Aula2

## Revisão: Aula1

In [6]:
import pandas as pd
dados = pd.read_csv(r'C:\Users\Helio\Documents\ALL THAT MATTERS\5 SEMESTRE - EESP\MACHINE LEARNING\Aula 2\train.csv')
dados.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,...,3,1,1,3,0,0,0,1,1,0


In [ ]:
sub_df = dados.drop(['FEMALE','NONWHITE','UNION','AGE','LNWAGE'], axis='columns')
sub_df['y'] = (sub_df['WAGE'] > 15).astype(int)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
y = sub_df['y'].values
X = sub_df.drop(['y','WAGE'], axis=1).values

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X,y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42, stratify=y)

In [ ]:
knn.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import classification_report
y_pred = knn.predict(X_test)
print(classification_report(y_test, y_pred))

## Regressão Logística

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(X_train,y_train)
y_pred = log_reg.predict(X_test)
print(classification_report(y_test, y_pred))

## Cross-Validation

In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_score

In [ ]:
cv_scores = cross_val_score(knn, X, y, cv=5)
print(cv_scores)
print("Média dos Scores CV (K-NN): {}".format(np.mean(cv_scores)))

In [ ]:
cv_scores = cross_val_score(log_reg, X, y, cv=5)
print(cv_scores)
print("Média dos Scores CV (Reg. Logística): {}".format(np.mean(cv_scores)))

In [ ]:
from sklearn.model_selection import cross_validate

In [ ]:
cv_metrics = cross_validate(knn, X, y, cv=5, scoring=('precision', 'recall'))
cv_metrics

In [ ]:
print("Média das Precisions CV (K-NN): {}".format(np.mean(cv_metrics['test_precision'])))
print("Média das Recalls CV (K-NN): {}".format(np.mean(cv_metrics['test_recall'])))

In [ ]:
cv_metrics = cross_validate(log_reg, X, y, cv=5, scoring=('precision', 'recall'))
cv_metrics

In [ ]:
print("Média das Precisions CV (Reg. Logística): {}".format(np.mean(cv_metrics['test_precision'])))
print("Média das Recalls CV (Reg. Logística): {}".format(np.mean(cv_metrics['test_recall'])))

## Trade-off entre Precision e Recall

In [ ]:
y_pred = log_reg.predict_proba(X_test)[:,1]

In [ ]:
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(y_test, y_pred)

print('Média de precision-recall: {0:0.2f}'.format(
      average_precision))

In [ ]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
%matplotlib inline

precision, recall, thresholds = precision_recall_curve(y_test, y_pred)

plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, step='post', alpha=0.2,
                 color='b')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('Trade-off entre Precision e Recall: AP={0:0.2f}'.format(
          average_precision))

In [ ]:
from sklearn.metrics import roc_curve, auc

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred)

In [ ]:
roc_auc = auc(fpr, tpr)
roc_auc

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='red',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC')
plt.legend(loc="lower right")
plt.show()

In [ ]:
y_pred_knn = knn.predict_proba(X_test)[:,1]

In [ ]:
fpr_knn, tpr_knn, thresholds = roc_curve(y_test, y_pred_knn)

In [ ]:
roc_auc_knn = auc(fpr_knn, tpr_knn)

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='red',
         lw=lw, label='ROC Reg. Log. (area = %0.2f)' % roc_auc)
plt.plot(fpr_knn, tpr_knn, color='blue',
         lw=lw, label='ROC K-NN (area = %0.2f)' % roc_auc_knn)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC')
plt.legend(loc="lower right")
plt.show()